In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
 

In [4]:
data_train = pd.read_csv('../data/processed/train_FD001.txt', sep=' ', header=None)
data_test = pd.read_csv('../data/processed/test_FD001.txt', sep=' ', header=None)
data_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1,1,0.459770,0.166667,0.183735,0.406802,0.309757,1.0,0.726248,0.242424,0.109755,0.369048,0.633262,0.205882,0.199608,0.363986,0.333333,0.713178,0.724662
1,1,2,0.609195,0.250000,0.283133,0.453019,0.352633,1.0,0.628019,0.212121,0.100242,0.380952,0.765458,0.279412,0.162813,0.411312,0.333333,0.666667,0.731014
2,1,3,0.252874,0.750000,0.343373,0.369523,0.370527,1.0,0.710145,0.272727,0.140043,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375
3,1,4,0.540230,0.500000,0.343373,0.256159,0.331195,1.0,0.740741,0.318182,0.124518,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386
4,1,5,0.390805,0.333333,0.349398,0.257467,0.404625,1.0,0.668277,0.242424,0.149960,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502


In [6]:
data_train.rename(columns={0: 'ID', 1: 'Cycles'}, inplace=True)
data_train["RUL"] = data_train.groupby("ID")["Cycles"].transform(lambda x: x.max() - x)
data_test.rename(columns={0: 'ID', 1: 'Cycles'}, inplace=True)
data_test["RUL"] = data_test.groupby("ID")["Cycles"].transform(lambda x: x.max() - x)
data_train.head()

,ID,Cycles,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,RUL
0,1,1,0.459770,0.166667,0.183735,0.406802,0.309757,1.0,0.726248,0.242424,0.109755,0.369048,0.633262,0.205882,0.199608,0.363986,0.333333,0.713178,0.724662,191
1,1,2,0.609195,0.250000,0.283133,0.453019,0.352633,1.0,0.628019,0.212121,0.100242,0.380952,0.765458,0.279412,0.162813,0.411312,0.333333,0.666667,0.731014,190
2,1,3,0.252874,0.750000,0.343373,0.369523,0.370527,1.0,0.710145,0.272727,0.140043,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375,189
3,1,4,0.540230,0.500000,0.343373,0.256159,0.331195,1.0,0.740741,0.318182,0.124518,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386,188
4,1,5,0.390805,0.333333,0.349398,0.257467,0.404625,1.0,0.668277,0.242424,0.149960,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502,187


In [16]:
print(data_train[["RUL"]].min(),data_train[["RUL"]].max(),data_train[["RUL"]].mean())

RUL    0
dtype: int64 RUL    361
dtype: int64 RUL    107.807862
dtype: float64


In [22]:

window_size = 30
def create_Sets(data_train, window_size):
    X = []
    Y = []
    # Agrupamos por ID de motor
    for j, group in data_train.groupby("ID"):
        group = group.sort_values("Cycles")
        features = group.iloc[:, 2:-1].values  # columnas 2 hasta penúltima (sin ID, Cycles, ni RUL)
        rul_values = group["RUL"].values
        
        if len(group) >= window_size:
            for i in range(len(group) - window_size + 1):
                X.append(features[i:i + window_size])
                Y.append(rul_values[i + window_size - 1])
        else:
            print(f"Warning: Unit {j} has less than {window_size} cycles")
    X = np.array(X)
    Y = np.array(Y)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(Y[:, None], dtype=torch.float32)
X_train, Y_train = create_Sets(data_train, window_size) 
X_test, Y_test = create_Sets(data_test, window_size) 

In [23]:
Y_train.shape

torch.Size([17731, 1])

In [24]:
class Predictions(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=17, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)
    def forward(self, x):
        x, _ = self.lstm(x)
        # extract only the last time step
        x = x[:, -1, :]
        x = self.dropout(x)
        x = self.linear(x)
        return x

In [25]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

model = Predictions().to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()

X_train = X_train.to(device)
Y_train = Y_train.to(device)
X_test = X_test.to(device)
Y_test = Y_test.to(device)

loader = data.DataLoader(data.TensorDataset(X_train, Y_train), shuffle=True, batch_size=8)

n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = torch.sqrt(loss_fn(y_pred, Y_train)).item()
        y_pred = model(X_test)
        test_rmse = torch.sqrt(loss_fn(y_pred, Y_test)).item()
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

Usando dispositivo: cpu
Epoch 0: train RMSE 63.5278, test RMSE 49.3951
Epoch 1: train RMSE 40.9871, test RMSE 45.6076
Epoch 2: train RMSE 36.8801, test RMSE 45.5412
Epoch 3: train RMSE 33.3499, test RMSE 54.0674
Epoch 4: train RMSE 31.9285, test RMSE 50.1126
Epoch 5: train RMSE 30.9069, test RMSE 49.1786
Epoch 6: train RMSE 31.0170, test RMSE 49.6661
Epoch 7: train RMSE 30.6399, test RMSE 50.2926
Epoch 8: train RMSE 30.4820, test RMSE 53.0578
Epoch 9: train RMSE 30.2342, test RMSE 51.0549
Epoch 10: train RMSE 30.5898, test RMSE 52.6680
Epoch 11: train RMSE 31.5044, test RMSE 55.6143
Epoch 12: train RMSE 31.0242, test RMSE 55.1712
Epoch 13: train RMSE 29.7763, test RMSE 53.8219
Epoch 14: train RMSE 29.6835, test RMSE 49.4288
Epoch 15: train RMSE 30.1368, test RMSE 54.6869
Epoch 16: train RMSE 29.9295, test RMSE 49.1723
Epoch 17: train RMSE 29.7743, test RMSE 51.6172
Epoch 18: train RMSE 30.5621, test RMSE 46.7371
Epoch 19: train RMSE 29.7645, test RMSE 50.9616
Epoch 20: train RMSE 29.71